In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q datasets evaluate accelerate peft bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, pipeline, AutoTokenizer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from huggingface_hub import notebook_login
from datasets import load_dataset
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import evaluate
from peft import get_peft_model, LoraConfig, TaskType
import torch
import os

sns.set_theme()
import pprint

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/W266/huggingface_credentials.txt", "r") as f:
    HF_TOKEN = f.read()

In [ ]:
dataset = load_dataset("jordanfan/processed_us_congress_117_bills_v3")
dataset_test = dataset['test']
dataset_test

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11277 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/3388 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/377 [00:00<?, ? examples/s]

Dataset({
    features: ['Unnamed: 0', 'index', 'id', 'policy_areas', 'cur_summary', 'cur_text', 'title', 'titles_official', 'titles_short', 'sponsor_name', 'sponsor_party', 'sponsor_state', 'cleaned_summary', 'extracted_text', 'extracted_text_375', 'extracted_text_750', 'extracted_text_1000', 'bertsum_extracted_250', 'bertsum_extracted_375', 'bertsum_extracted_375_1000', 'bertsum_extracted_250_1000', 'bertsum_extracted_375_750', 'bertsum_extracted_250_750', 'bertsum_extracted_375_500', 'bertsum_extracted_250_500', 'bertsum_extracted_375_375', 'bertsum_extracted_250_375'],
    num_rows: 377
})

In [ ]:
# ====== ENVIRONMENT ======
DEV = False
EXPLORE = False
PEFT = False
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# ====== DATA =======
INPUT_MAX_LENGTH = 512
LABEL_MAX_LENGTH = 128
INPUT_COLUMN = "cur_text"
LABEL_COLUMN = "cleaned_summary"

# ====== MODEL ======
CHECKPOINT = "facebook/bart-large"
PATH = '/content/drive/MyDrive/colab-notebooks/W266/'
MODEL_NAME = "bart_baseline_peft"
#else:
#  MODEL_NAME = "pegasus-legalease"
HUGGINGFACE_DIR = "jordanfan/" + MODEL_NAME
CUSTOM_NAME = "bart_baseline_peft"

# ====== OPTIIZER =======
OPTIMIZER = "adamw_torch"
LEARNING_RATE = 2e-4
WEIGHT_DECAY = 0.01

# ====== TRAINING ======
EVAL_STRATEGY = "steps"
BATCH_SIZE = 8
EPOCHS = 2
SAVE_STEPS = 100 if DEV else 1000
EVAL_STEPS = 100 if DEV else 250
LOGGING_STEPS = 100 if DEV else 500
EARLY_STOPPING_PATIENCE = 3
EARLY_STOPPING_THRESHOLD = 0.005

In [ ]:
dataset = load_dataset("jordanfan/processed_us_congress_117_bills_v3")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
model = AutoModelForSeq2SeqLM.from_pretrained(CHECKPOINT)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [ ]:
# Tokenize the entire dataset
def tokenize_function(examples):
	model_inputs = tokenizer(examples[INPUT_COLUMN], return_tensors="pt", max_length=INPUT_MAX_LENGTH, padding=True, truncation=True)
	labels = tokenizer(text_target=examples[LABEL_COLUMN], max_length=LABEL_MAX_LENGTH, padding=True, truncation=True)
	model_inputs["labels"] = labels["input_ids"]

	return model_inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=CHECKPOINT)

Map:   0%|          | 0/11277 [00:00<?, ? examples/s]

Map:   0%|          | 0/3388 [00:00<?, ? examples/s]

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

In [ ]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)

modules = find_target_modules(model)
print(modules)

['fc2', 'lm_head', 'v_proj', 'fc1', 'out_proj', 'k_proj', 'q_proj']


In [ ]:
if PEFT:
  peft_config = LoraConfig(
      task_type=TaskType.SEQ_2_SEQ_LM,
      inference_mode=False,
      target_modules=modules,
      r=16,
      lora_alpha=32,
      lora_dropout=0.1
  )

  model = get_peft_model(model, peft_config)
  model.print_trainable_parameters()

In [ ]:
from transformers import EarlyStoppingCallback

early_stopping = EarlyStoppingCallback(early_stopping_patience=EARLY_STOPPING_PATIENCE, early_stopping_threshold=EARLY_STOPPING_THRESHOLD)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=HUGGINGFACE_DIR,
    evaluation_strategy=EVAL_STRATEGY,
    save_strategy=EVAL_STRATEGY,
    save_steps=SAVE_STEPS,
    eval_steps=EVAL_STEPS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=WEIGHT_DECAY,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    num_train_epochs=EPOCHS,
    # predict_with_generate=True,
    fp16=True,
    report_to=["tensorboard"],
    push_to_hub=True,
    hub_token=HF_TOKEN
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
    callbacks = [early_stopping]
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss
250,No log,1.471109
500,1.858800,1.400453
750,1.858800,1.303574
1000,1.409200,1.262865
1250,1.409200,1.201145
1500,1.250200,1.157974
1750,1.250200,1.138654
2000,0.981500,1.104506
2250,0.981500,1.070992
2500,0.921300,1.034666


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


Step,Training Loss,Validation Loss
250,No log,1.471109
500,1.858800,1.400453
750,1.858800,1.303574
1000,1.409200,1.262865
1250,1.409200,1.201145
1500,1.250200,1.157974
1750,1.250200,1.138654
2000,0.981500,1.104506
2250,0.981500,1.070992
2500,0.921300,1.034666


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=2820, training_loss=1.2373572166929854, metrics={'train_runtime': 2577.1887, 'train_samples_per_second': 8.751, 'train_steps_per_second': 1.094, 'total_flos': 2.4438438598017024e+16, 'train_loss': 1.2373572166929854, 'epoch': 2.0})

In [ ]:
trainer.create_model_card(
    language='english',
    model_name=MODEL_NAME,
    finetuned_from=CHECKPOINT,
    tasks='summarization',
    tags='summarization',
    dataset='hheiden/us-congress-117-bills',
    dataset_args=f"Max token input: {INPUT_MAX_LENGTH} | {LABEL_MAX_LENGTH}"
)

In [ ]:
if DEV:
  commit_msg = f"training completed[dev]: {CUSTOM_NAME}"
else:
  commit_msg = f"training completed[prod]: {CUSTOM_NAME}"

trainer.push_to_hub(commit_message=commit_msg)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


events.out.tfevents.1712084136.b649ef780a74.217.0:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jordanfan/bart_baseline_peft/commit/5396c928604768781eb1fe3df827a402b71b7fd2', commit_message='training completed[prod]: bart_baseline_peft', commit_description='', oid='5396c928604768781eb1fe3df827a402b71b7fd2', pr_url=None, pr_revision=None, pr_num=None)